# Especificar una expresión regular para la coincidencia mas corta  

2.7

- Problema

        Intenta hacer coincidir un patrón de texto con expresiones regulares, pero está identificando el coincidencias más largas posibles de un patrón. En cambio, le gustaría cambiarlo para encontrar el coincidencia más corta posible.    
        
          
- Solución

        Este problema a menudo surge en patrones que intentan hacer coincidir el texto encerrado dentro de un par de delimitadores inicial y final (por ejemplo, una cadena entre comillas).   
       
Para ilustrarlo, considere este ejemplo:

In [11]:
import re

In [2]:
str_pat = re.compile(r'\"(.*)\"')

In [3]:
text1 = 'Computer says "no."'

In [4]:
str_pat.findall(text1)

['no.']

In [5]:
text2 = 'Computer says "no." Phone says "yes."'

In [14]:
str_pat.findall(text2)

['no.', 'yes.']

En este ejemplo, el patrón `r'\ "(. *) \"`  Está intentando hacer coincidir el texto incluido dentro
citas.   
Sin embargo, el operador `*` en una expresión regular es codicioso, por lo que la coincidencia se basa
en encontrar la coincidencia más larga posible.  
Por tanto, en el segundo ejemplo que involucra text2, coincide incorrectamente con las dos cadenas entre comillas.  
Para solucionar esto, agregue el `?` después del operador `*` en el patrón, así:

In [24]:
str_pat = re.compile(r'\"(.*?)\"') 
# el punto busca cualquier caracter exepto salto de linea

In [6]:
str_pat.findall(text2)

['no." Phone says "yes.']

In [7]:
text3 = 'Computer says "no."\nPhone says "yes."'

In [8]:
str_pat1 = re.compile(r'\"(.*)\"')
str_pat2 = re.compile(r'\"(.*?)\"') 

In [9]:
print(text3)
print(str_pat1.findall(text3))
print(str_pat2.findall(text3))

Computer says "no."
Phone says "yes."
['no.', 'yes.']
['no.', 'yes.']


In [10]:
print(text2)
print(str_pat1.findall(text2))
print(str_pat2.findall(text2))

Computer says "no." Phone says "yes."
['no." Phone says "yes.']
['no.', 'yes.']


# Escribir una expresión regular para patrones de varias líneas  

2.8

- Problema

        Estás intentando hacer coincidir un bloque de texto con una expresión regular, pero necesitas la coincidencia para abarcar varias líneas.  
          
          
- Solución

        Este problema suele surgir en patrones que utilizan el punto (.) Para coincidir con cualquier carácter. Olvídese de tener en cuenta el hecho de que no coincide con las nuevas líneas.     
        
Por ejemplo, suponga que están tratando de hacer coincidir los comentarios de estilo C:

In [12]:
comment = re.compile(r'/\*(.*?)\*/')

In [13]:
text1 = '/* this is a comment */'
text2 = """/* this is a
multiline comment */"""

In [14]:
print(text2)

/* this is a
multiline comment */


In [15]:
comment.findall(text1)

[' this is a comment ']

In [16]:
comment.findall(text2)

[]

In [17]:
# Para solucionar el problema, 
# puede agregar soporte para nuevas líneas.
comment = re.compile(r'/\*((?:.|\n)*?)\*/')

In [18]:
comment.findall(text2)

[' this is a\nmultiline comment ']

En este patrón, `(?:. | \n)` especifica un grupo de no captura (es decir, define un grupo para el
propósitos de emparejamiento, pero ese grupo no se captura por separado ni se numera).

La función re.compile () acepta una bandera, `re.DOTALL`, que es útil aquí. Hace
la `.` en una expresión regular, coincida con todos los caracteres, incluidas las nuevas líneas.   
Por ejemplo:

In [19]:
comentario = re.compile (r'/\*(.*?)\*/', re.DOTALL)

In [20]:
comentario.findall(text2)

[' this is a\nmultiline comment ']

El uso de la marca re.DOTALL funciona bien para casos simples, pero puede ser problemático si está
trabajar con patrones extremadamente complicados o una combinación de expresiones regulares independientes
que se han combinado con el propósito de tokenizar.   
Si se le da una opción, generalmente es mejor definir su patrón de expresión regular
para que funcione correctamente sin necesidad de banderas adicionales.

# Normalizar el texto Unicode a una representación estandar

2.9

- Problema

        Está trabajando con cadenas Unicode, pero debe asegurarse de que todas las cadenas tengan la misma representación subyacente.   
        
  
- Solución

        En Unicode, ciertos caracteres se pueden representar mediante más de una secuencia válida de puntos de código. 
        
Para ilustrarlo, considere el siguiente ejemplo:

In [21]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'

In [22]:
s1

'Spicy Jalapeño'

In [23]:
s2

'Spicy Jalapeño'

In [24]:
s1 == s2

False

In [25]:
len(s1)

14

In [26]:
len(s2)

15

Aquí se ha presentado el texto “Jalapeño picante” en dos formas. El primero usa el
compuesto de carácter “ñ” (U + 00F1). El segundo usa la letra latina "n" seguida de una
Carácter de combinación “~” (U + 0303).  
Tener múltiples representaciones es un problema para los programas que comparan cadenas. 
Para solucionar este problema, primero debe normalizar el texto en una representación estándar usando
el módulo unicodedata:

In [28]:
import unicodedata

In [29]:
t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)
t1 == t2

True

In [30]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
t3 == t4

True

In [31]:
print(ascii(t3))
print(t3)

'Spicy Jalapen\u0303o'
Spicy Jalapeño


El primer argumento para `normalize()` especifica cómo desea normalizar la cadena.   
- `NFC` significa que los caracteres deben estar completamente compuestos (es decir, usar un solo punto de código si es posible).  
- `NFD` significa que los caracteres deben descomponerse completamente con el uso de caracteres combinados.  
Python también admite las formas de normalización `NFKC` y `NFKD`, que añaden extra de compatibilidad para tratar con ciertos tipos de personajes.   
Por ejemplo:

In [32]:
s = '\ufb01' # solo un caracter
s

'ﬁ'

In [33]:
unicodedata.normalize('NFD', s)

'ﬁ'

In [34]:
# Observe cómo las letras combinadas se dividen aquí.
unicodedata.normalize('NFKD', s)

'fi'

In [35]:
unicodedata.normalize('NFKC', s)

'fi'

La normalización es una parte importante de cualquier código que necesita asegurarse de que procesa Texto Unicode de una manera sana y coherente.  
Esto es especialmente cierto al procesar cadenas recibido como parte de la entrada del usuario donde tiene poco control de la codificación.  
La normalización también puede ser una parte importante de la desinfección y el filtrado del texto.   
Por ejemplo, suponga que desea eliminar todos los signos diacríticos de algún texto (posiblemente con el propósito de una busqueda o emparejar el texto):

In [36]:
t1 = unicodedata.normalize('NFD', s1)
''.join(c for c in t1 if not unicodedata.combining(c))

'Spicy Jalapeno'

In [37]:
texto="Hola soy ëmiláno đ€ argentinæ"
t1 = unicodedata.normalize('NFD', texto)
''.join(c for c in t1 if not unicodedata.combining(c))

'Hola soy emilano đ€ argentinæ'

In [39]:
texto="Hola soy ëmiláno đ€ argentiná"
t1 = unicodedata.normalize('NFKD', texto)
''.join(c for c in t1 if not unicodedata.combining(c))

'Hola soy emilano đ€ argentina'

Este último ejemplo muestra otro aspecto importante del módulo unicodedata, a saber,
funciones de utilidad para probar caracteres contra clases de caracteres. La función de combining()
prueba un carácter para ver si es un carácter de combinación. Hay otras funciones en
el módulo para encontrar categorías de caracteres, probar dígitos, etc.
Unicode es obviamente un tema extenso. Para obtener información de referencia más detallada sobre normalización, visite la página de Unicode sobre el tema http://www.unicode.org/faq/normalization.html  

In [40]:
import unicodedata

def limpiar_texto(texto):
    # Normalizar el texto a la forma NFD (Normalization Form Decomposition)
    texto_normalizado = unicodedata.normalize('NFD', texto)
    # Eliminar los caracteres de combinación (acentos, tildes, etc.)
    texto_limpio = ''.join(c for c in texto_normalizado if not unicodedata.combining(c))
    return texto_limpio


In [41]:

# Ejemplo de uso
texto = 'Hola soy ëmiláno đ€ argentiná'
texto_limpio = limpiar_texto(texto)
print(texto_limpio)

Hola soy emilano đ€ argentina


# Trabajar con caracteres Unicode en formato normal con Expresiones Regulares

2.10

- Problema
        
        Utiliza expresiones regulares para procesar texto, pero le preocupa el manejo de caracteres Unicode.    
        

- Solución
        
        De forma predeterminada, el módulo re ya está programado con conocimientos rudimentarios de ciertas clases de caracteres Unicode.
        Por ejemplo, `\d` ya coincide con cualquier dígito Unicode
        

In [43]:
import re
num = re.compile('\d+')
# ASCII digits
num.match('123')

<re.Match object; span=(0, 3), match='123'>

In [44]:
# Arabic digits
print('\u0661\u0662\u0663')
num.match('\u0661\u0662\u0663')

١٢٣


<re.Match object; span=(0, 3), match='١٢٣'>

Si necesita incluir caracteres Unicode específicos en patrones, puede usar el habitual
secuencia de escape para caracteres Unicode (por ejemplo, \ uFFFF o \ UFFFFFFF). Por ejemplo, aquí
es una expresión regular que coincide con todos los caracteres en algunas páginas de códigos árabes diferentes:
```python
árabe = re.compile ('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+')
```
Al realizar operaciones de búsqueda y búsqueda, es una buena idea normalizar y
posiblemente primero desinfecte todo el texto a una forma estándar . Sin embargo, también es importante estar al tanto de casos especiales.   
Por ejemplo, considere el comportamiento de la coincidencia entre mayúsculas y minúsculas:

In [45]:
pat = re.compile('stra\u00dfe', re.IGNORECASE)
s = 'straße'
pat.match(s)

<re.Match object; span=(0, 6), match='straße'>

In [46]:
print(pat.match(s.upper())) # sin coincidencias

None


In [47]:
s.upper()

'STRASSE'

Mezclar Unicode y expresiones regulares es a menudo una buena manera de hacer explotar su cabeza.
Si va a hacerlo en serio, debería considerar instalar regex http://pypi.python.org/pypi/regex , que proporciona soporte completo para plegado de casos Unicode, así como una variedad de otros
características interesantes, incluida la coincidencia aproximada.

# Eliminación de caracteres no deseados 

2.11

- Problema

        Desea eliminar los caracteres no deseados, como los espacios en blanco, desde el principio, el final o en medio de una cadena de texto.  
        

- Solución

        El método strip() se puede utilizar para quitar caracteres desde el principio o el final de un string. lstrip() y rstrip() realizan la eliminación desde el lado izquierdo o derecho, respectivamente.  
        De forma predeterminada, estos métodos eliminan los espacios en blanco, pero se pueden dar otros caracteres.   

Por ejemplo:

In [48]:
s = '     hello world \n'
s.strip()

'hello world'

In [49]:
s.lstrip()

'hello world \n'

In [50]:
s.rstrip()

'     hello world'

In [51]:
t = '-----hello====='
t.lstrip('-')

'hello====='

In [52]:
>>> t.strip('-=') 

'hello'

In [53]:
s = ' hola      mundo'
s = s.strip()
s

'hola      mundo'

Si necesitara hacer algo en el espacio interior, necesitaría usar otra tecnología, como usar el método replace() o una sustitución de expresión regular.  
Por ejemplo

In [54]:
s.replace(' ', '')

'holamundo'

In [55]:
re.sub('\s+', ' ', s)

'hola mundo'

Suele ocurrir que desee combinar operaciones de eliminación de string con otras
tipo de procesamiento iterativo, como leer líneas de datos de un archivo. Si es así, este es uno área donde una expresión generadora puede ser útil.   
Por ejemplo:
```python
with open (nombre de archivo) como f:
    lines = (line.strip() for line in f)
    for línea en línes:
        ...
```
Aquí, la expresión lines = `(line.strip() for línea en f)` actúa como un tipo de datos generator.  
Es eficiente porque en realidad no lee primero los datos en ningún tipo de lista temporal.  
Simplemente crea un iterador donde a todas las líneas producidas se les aplica la operación de eliminación.
Para una eliminación aún más avanzada, puede recurrir al método translate().

# Desinfectar y limpiar el texto

2.12

- Problema

        Algún niño aburrido de guiones ha ingresado el texto "pýtĥöñ" en un formulario en su página web y te gustaría limpiarlo de alguna manera.  
        
        
- Solución

        El problema de desinfectar y limpiar el texto se aplica a una amplia variedad de problemas que implican el análisis de texto y el manejo de datos. En un nivel muy simple, puede usar básico funciones de cadena (por ejemplo, str.upper () y str.lower ()) para convertir texto en un caso estándar.
        Los reemplazos simples que utilizan str.replace () o re.sub () pueden centrarse en eliminar o cambiar secuencias de caracteres muy específicas. También puede normalizar el texto usando unicode data.normalize(), como se muestra en la Receta 2.9.
        Sin embargo, es posible que desee llevar el proceso de saneamiento un paso más allá. Quizás, por Por ejemplo, desea eliminar rangos completos de caracteres o eliminar marcas diacríticas.
        Si lo hace, puede recurrir al método str.translate () que a menudo se pasa por alto. Para ilustrar, suponga que tiene una cadena desordenada como la siguiente:

In [56]:
s = 'pýtĥöñ\fis\tawesome\r\n'
print(s)

pýtĥöñis	awesome



El primer paso es limpiar el espacio en blanco. Para hacer esto, haga una pequeña tabla de traducción. y usa translate():

In [57]:
remap = {ord('\t') : ' ',ord('\f') : ' ',ord('\r') : None}

In [58]:
a = s.translate(remap)
a

'pýtĥöñ is awesome\n'

Como puede ver aquí, los caracteres de espacio en blanco como \t y \f se han reasignado a un único espacio. El retorno de carro \r se ha eliminado por completo.
Puede llevar esta idea de reasignación un paso más allá y crear tablas mucho más grandes.  
Por exemplo eliminemos todos los caracteres combinados:

In [59]:
import unicodedata
import sys
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
                         if unicodedata.combining(chr(c)))

In [60]:
print(s)
b = unicodedata.normalize('NFD', a)
b

pýtĥöñis	awesome



'pýtĥöñ is awesome\n'

In [61]:
b.translate(cmb_chrs)

'python is awesome\n'

Como otro ejemplo, aquí hay una tabla de traducción que asigna todos los dígitos decimales Unicode
caracteres a su equivalente en ASCII:

In [62]:
digitmap = { c : ord('0') + unicodedata.digit(chr(c)) for c in range(sys.maxunicode)
            if unicodedata.category(chr(c)) == 'Nd' }

In [63]:
len(digitmap)

650

In [66]:
x = '\u0661\u0662\u0663' # Arabic digits
print(x)
x.translate(digitmap)

١٢٣


'123'

Otra técnica más para limpiar texto implica la decodificación de E/S y la función de codificación. La idea aquí es primero hacer una limpieza preliminar del texto y luego ejecutarlo mediante una combinación de operaciones encode() o decode() para eliminarlo o alterarlo.  
por ejemplo:

In [67]:
print(a)
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

pýtĥöñ is awesome



'python is awesome\n'

Aquí, el proceso de normalización descompuso el texto original en caracteres junto con
caracteres de combinación separados. La codificación / decodificación ASCII posterior simplemente descartó todos esos caracteres de una sola vez. Naturalmente, esto solo funcionaría si la representación ASCII era el objetivo final.

Un problema importante con la desinfección del texto puede ser el rendimiento en tiempo de ejecución. Como regla general, la
cuanto más simple sea, más rápido se ejecutará. Para reemplazos simples, el método str.replace ()
suele ser el método más rápido, incluso si tiene que llamarlo varias veces. Por ejemplo, para
limpiar los espacios en blanco, puede usar un código como este:

```python
def clean_spaces(s):
    s = s.replace('\r', '')
    s = s.replace('\t', ' ')
    s = s.replace('\f', ' ')
    return s
```
Si lo prueba, encontrará que es un poco más rápido que usar translate () o un enfoque
usando una expresión regular.
Por otro lado, el método translate () es muy rápido si necesita realizar cualquier
tipo de reasignación o eliminación no trivial de carácter a carácter.

# Alinear cadenas de texto
2.13

- Problema

        Necesita formatear el texto con algún tipo de alineación aplicada.  


- Solución

        Para la alineación básica de cadenas, los métodos de cadenas ljust (), rjust () y center() puede ser usado.  
        
Por ejemplo:

In [68]:
text = 'Hello World'

In [69]:
text.ljust(20)

'Hello World         '

In [70]:
text.rjust(20)

'         Hello World'

In [71]:
text.center(20)

'    Hello World     '

In [72]:
text.rjust(20,'=')

'=========Hello World'

In [11]:
text.center(20,'*')

'****Hello World*****'

La función format () también se puede utilizar para alinear cosas fácilmente. Todo lo que necesitas hacer es usar los caracteres <,> o ^ junto con el ancho deseado.   
Por ejemplo:

In [73]:
format(text, '=>20s')

'=========Hello World'

In [74]:
format(text, '*^20s')

'****Hello World*****'

In [75]:
'{:>10s} {:>10s}'.format('Hello', 'World')

'     Hello      World'

In [76]:
x = 1.2345
format(x, '>10')

'    1.2345'

In [77]:
format(x, '^10.2f')

'   1.23   '